In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
from datetime import date

In [48]:
df = pd.read_parquet('../../data/wrds/options_prices_2020_2023.parquet')

prices = yf.download('^GSPC',start = date(2019,1,1),end = date(2023,8,31))['Close']
returns = prices.pct_change().dropna()
vol = returns.rolling(252).std() * np.sqrt(252)
vol.index = pd.to_datetime(vol.index).date
vol = vol[vol.index >= date(2020,1,1)]

[*********************100%***********************]  1 of 1 completed


In [50]:
filter = ((df['secid'] == 108105) &     # SPX 
          (df['cp_flag'] == 'P')  &     # Puts
        (df['delta'] >= -0.51) & (df['delta'] <= -0.49)  # ATM
)

df = (df.loc[filter,:]
 .sort_values(by = ['date','delta'])
 .drop_duplicates(subset=['date'],keep='first')
)

In [51]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = df['date'],
        y = df['best_bid'],
        name = 'bid',
        opacity=1,
    )
)

fig.add_trace(
    go.Scatter(
        x = df['date'],
        y = df['best_offer'],
        name = 'offer',
        opacity= 1,
        line=dict(dash='dot')
    ),
)

fig.update_layout(title = 'SPX Put Option Prices (ATM, 1M)')
fig.update_yaxes(title = 'Price')
fig.show()

In [55]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = df['date'],
        y = df['impl_volatility'],
        name = 'implied vol',
        showlegend=True,
        opacity=1,
    )
)

fig.update_layout(title = 'SPX Put IV (ATM, 1M)')
fig.update_yaxes(title = 'IV')
fig.show()